# Dissertation Scraper

Procedure:
1. Export dissertation meta-data using proquest into CSV/Excel
2. For each link in that spreadsheet
    3. Go to Proquest page
    4. Find download link (if exists)
    5. Download PDF to output folder
    
 

In [11]:
# Common webscraping libaries
from bs4 import BeautifulSoup as bs
from markdownify import markdownify
from urllib.parse import urlparse
from tqdm.notebook import tqdm
# Data manipulation libraries
import pandas as pd
import numpy as np
import requests
import time
import re
tqdm.pandas()

In [15]:
auth_url = "https://www.proquest.com/docview/2550299969"
url = "https://www.proquest.com/docview/2550299969"
url

'https://www.proquest.com/docview/2550299969'

In [17]:
r = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0'})
soup = bs(r.text,
          features="html.parser")
soup

<!DOCTYPE html>
<html dir="ltr" lang="en" xmlns="http://www.w3.org/1999/xhtml"><head><meta content="Apache Tapestry Framework (version 5.3.8)" name="generator"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="Explore millions of resources from scholarly journals, books, newspapers, videos and more, on the ProQuest Platform." name="description"/><meta content="noindex,nofollow" name="robots"/><link href="/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/><link href="/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/><link href="/apple-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/><link href="/apple-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/><link href="/apple-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/><link href="/apple-icon-120x120.png" rel="apple-touch-icon" sizes="120x120"/><link href="/apple-icon-144x144.png" rel="apple-touch-icon" sizes="14

In [23]:
from selenium import webdriver
from creds import *
driver = webdriver.Chrome('/Users/cameron/Desktop/Work/Princeton/Dissertation_Scraper/chromedriver')

In [24]:
from pathlib import Path
# from math import random
def get_pdf(driver,url):
    time.sleep(1)
    driver.get(url)
    title = url
    try:
        title = driver.find_element_by_id("documentTitle").text
    except:
        print(f"No title for {url}")
    title = title.replace(":","_").replace("/","_").strip().replace(" ","_")
    filename = Path(f"data/output/{title}.pdf")
    try:
        pdf_url = driver.find_element_by_id("downloadPDFLink").get_attribute('href')
        r = requests.get(pdf_url,headers = {'User-Agent': 'Mozilla/5.0'})
        filename.write_bytes(r.content)
    except Exception as e:
        print(f"{title} Error Message: {e}")
        return False
    return True
    

In [21]:
get_pdf(driver,url)

True

In [ ]:
url_df = pd.read_excel("data/input/ProQuestDocuments-2021-07-13.xls")
# Get rid of account ID
url_df["DocumentURL"] = url_df["DocumentURL"].str.split("?").str[0]
url_df["DocumentURL"].progress_map(lambda dis_url : get_pdf(driver,dis_url))

  0%|          | 0/100 [00:00<?, ?it/s]

American_Dreams__How_Deferred_Action_for_Childhood_Arrivals_Shaped_the_Politics_of_Immigraiton_Policy Error Message: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="downloadPDFLink"]"}
  (Session info: chrome=91.0.4472.164)

No title for https://www.proquest.com/dissertations-theses/attendance-race-teachers-perceptions-parental/docview/2550291241/se-2
https___www.proquest.com_dissertations-theses_attendance-race-teachers-perceptions-parental_docview_2550291241_se-2 Error Message: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="downloadPDFLink"]"}
  (Session info: chrome=91.0.4472.164)

No title for https://www.proquest.com/dissertations-theses/examination-high-school-teachers-perceptions/docview/2550291235/se-2
https___www.proquest.com_dissertations-theses_examination-high-school-teachers-perceptions_docview_2550291235_se-2 Error Message: Message: no such element: Unable to locate element: {"met

In [8]:
driver.quit()